In [1]:
import os
import cv2

import numpy as np
import pandas as pd

import pickle

from imblearn.over_sampling import SMOTE

In [2]:
### THIS HAS ALREADY BEEN RUN .. NO NEED TO RUN AGAIN ###

# with open('lbp/df_pos_ryan.pkl', 'rb') as f:
#     df_pos_ryan = pickle.load(f)

# with open('lbp/df_pos_pt.pkl', 'rb') as f:
#     df_pos_pt = pickle.load(f)

# df_pos = pd.concat([df_pos_ryan, df_pos_pt])

# df_pos.head()

# enc_names = []
# enc_encodings = []
# for i, row in df_pos.iterrows():
#     enc_names.append(row['CLASS'])
#     enc_encodings.append(np.array(row[:-1]))

# encodings = {
#     'names':enc_names,
#     'encodings':enc_encodings
# }

# len(encodings['encodings'])

# for i in range(4481-87):
#     X, y = SMOTE().fit_resample(encodings['encodings'], encodings['names'])
#     encodings['encodings'].append(X[-1])
#     encodings['names'].append(y[-1])

In [3]:
with open('smoted_encodings_lbp.pickle', 'rb') as f:
    encodings = pickle.load(f)

In [4]:
len(encodings['encodings'])

4481

In [5]:
pd.Series(encodings['names']).value_counts()

PT      2241
Ryan    2240
dtype: int64

In [ ]:
# with open('smoted_encodings_lbp.pickle', 'wb') as f:
#     pickle.dump(encodings, f)

In [ ]:
# with open('smoted_encodings_lbp.pickle', 'rb') as f:
#     encodings = pickle.load(f)

### RE-TRAIN WITH MORE BALANCED CLASSES

In [7]:
with open('lbp/df_neg.pkl', 'rb') as f:
    df_neg = pickle.load(f)

In [8]:
df_neg.shape

(4481, 58)

In [9]:
df_pos = pd.DataFrame.from_dict(encodings)

In [10]:
df_pos.head()

,names,encodings
0,Ryan,"[1916, 214, 22, 18, 37, 14, 29, 153, 17, 20, 2..."
1,Ryan,"[1779, 149, 13, 29, 31, 13, 15, 137, 14, 16, 2..."
2,Ryan,"[2619, 274, 40, 21, 75, 17, 33, 219, 27, 16, 5..."
3,Ryan,"[2087, 175, 16, 14, 44, 20, 22, 140, 10, 25, 2..."
4,Ryan,"[1656, 135, 12, 15, 27, 19, 19, 118, 10, 13, 2..."


In [11]:
df_pos_list = pd.DataFrame(df_pos['encodings'].values.tolist())

In [12]:
df_pos_list['CLASS'] = df_pos['names']

In [13]:
df_pos_list.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,CLASS
0,1916.0,214.0,22.0,18.0,37.0,14.0,29.0,153.0,17.0,20.0,...,18.0,65.0,68.0,62.0,20.0,18.0,18.0,33.0,75.0,Ryan
1,1779.0,149.0,13.0,29.0,31.0,13.0,15.0,137.0,14.0,16.0,...,23.0,50.0,54.0,71.0,24.0,24.0,14.0,34.0,58.0,Ryan
2,2619.0,274.0,40.0,21.0,75.0,17.0,33.0,219.0,27.0,16.0,...,32.0,50.0,40.0,54.0,29.0,14.0,19.0,43.0,57.0,Ryan
3,2087.0,175.0,16.0,14.0,44.0,20.0,22.0,140.0,10.0,25.0,...,18.0,73.0,48.0,45.0,20.0,19.0,24.0,35.0,69.0,Ryan
4,1656.0,135.0,12.0,15.0,27.0,19.0,19.0,118.0,10.0,13.0,...,10.0,91.0,66.0,55.0,19.0,13.0,17.0,39.0,68.0,Ryan


In [14]:
df_all = pd.concat([df_pos_list, df_neg])

In [15]:
# Split X, y.
X = df_all.drop('CLASS', axis=1)
y = df_all['CLASS']

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)

In [34]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [35]:
y_pred = model.predict(X_test)

In [36]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
cm

array([[442,   0,   3],
       [  0, 450,   2],
       [  0,   0, 896]])

In [37]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          PT       1.00      0.99      1.00       445
        Ryan       1.00      1.00      1.00       452
     Unknown       0.99      1.00      1.00       896

    accuracy                           1.00      1793
   macro avg       1.00      1.00      1.00      1793
weighted avg       1.00      1.00      1.00      1793



In [38]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [39]:
print_cm(cm, labels=['PT', 'Ryan', 'Unknown'])

                 PT    Ryan Unknown 
         PT   442.0     0.0     3.0 
       Ryan     0.0   450.0     2.0 
    Unknown     0.0     0.0   896.0 
